# XGBoost

XGBoost(eXtreme Gradient Boosting)は、分布型勾配ブースティングフレームワークで、Tianqi Chenにより開発されました。このフレームワークは高い効率と拡張性を持ち、欠損値の処理、正則化、並列処理などの特徴を持っています。さらに、決定木アルゴリズムをベースとしており、大規模なデータセットに対しても高速に動作し、精度も高いことで知られています。

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/develop/tabledata/regression/XGBoost_sklearn.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xgboost

In [1]:
import xgboost as xgb

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

#### データの用意

In [3]:
california = fetch_california_housing()
df = pd.DataFrame(california.data, columns=california.feature_names)
df['target'] = pd.Series(california.target, name='MedHouseVal')
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [4]:
x = df.drop('target', axis = 1)
y = df['target']


x_train,x_valid,y_train,y_valid = train_test_split(x,y,test_size = 0.2, random_state=4)

#### XGBoostの学習

In [5]:
xgb_train = xgb.DMatrix(x_train, label=y_train)
xgb_valid = xgb.DMatrix(x_valid, label=y_valid)
evals = [(xgb_train,'train'),(xgb_valid,'eval')]

/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed 

In [6]:
# https://qiita.com/m_yama123/items/133e5b73b2e2bcd6bd6a
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'learning_rate':0.01,
}


model = xgb.train(
            params,
            xgb_train,
            num_boost_round=1000,
            early_stopping_rounds=100,
            evals=evals,
            verbose_eval=100
        )

[0]	train-rmse:1.14952	eval-rmse:1.13300
[100]	train-rmse:0.69895	eval-rmse:0.70570
[200]	train-rmse:0.55592	eval-rmse:0.58361
[300]	train-rmse:0.49596	eval-rmse:0.53492
[400]	train-rmse:0.45596	eval-rmse:0.50335
[500]	train-rmse:0.43590	eval-rmse:0.49087
[600]	train-rmse:0.41878	eval-rmse:0.48276
[700]	train-rmse:0.40403	eval-rmse:0.47619
[800]	train-rmse:0.39216	eval-rmse:0.47194
[900]	train-rmse:0.38132	eval-rmse:0.46812
[999]	train-rmse:0.37155	eval-rmse:0.46452


#### 予測

In [9]:
y_pred = model.predict(xgb.DMatrix(x_valid))

/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed 

In [10]:
# 平均二乗誤差を計算します
mse = mean_squared_error(y_valid, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.21577597562179798
